# Mean Absolute errors


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# do we have the correct env loaded?
assert 'donkey2' == os.environ['CONDA_DEFAULT_ENV'], os.environ['CONDA_DEFAULT_ENV']

In [4]:
import numpy as np
fast_model_fast_data_3d = np.load('open-loop-results/nighttime-fast-auto-3d-model-nighttime-fast-auto-data-prediction-and-ground-truth.npy')
fast_model_slow_data_3d = np.load('open-loop-results/nighttime-fast-auto-3d-model-nighttime-slow-auto-data-prediction-and-ground-truth.npy')
slow_model_fast_data_3d = np.load('open-loop-results/nighttime-slow-auto-3d-model-nighttime-fast-auto-data-prediction-and-ground-truth.npy')
slow_model_slow_data_3d = np.load('open-loop-results/nighttime-slow-auto-3d-model-nighttime-slow-auto-data-prediction-and-ground-truth.npy')
fast_model_fast_data_linear = np.loadtxt('open-loop-results/nighttime-fast-auto-linear-model-nighttime-fast-auto-data-prediction-and-ground-truth.csv.gz')
fast_model_slow_data_linear = np.loadtxt('open-loop-results/nighttime-fast-auto-linear-model-nighttime-slow-auto-data-prediction-and-ground-truth.csv.gz')
slow_model_fast_data_linear = np.loadtxt('open-loop-results/nighttime-slow-auto-linear-model-nighttime-fast-auto-data-prediction-and-ground-truth.csv.gz')
slow_model_slow_data_linear = np.loadtxt('open-loop-results/nighttime-slow-auto-linear-model-nighttime-slow-auto-data-prediction-and-ground-truth.csv.gz')


In [5]:
preds_and_ground_truths = dict(
    fast_fast_3d=fast_model_fast_data_3d,
    slow_fast_3d=slow_model_fast_data_3d,
    fast_fast_linear=fast_model_fast_data_linear,
    slow_fast_linear=slow_model_fast_data_linear,
    fast_slow_3d=fast_model_slow_data_3d,
    slow_slow_3d=slow_model_slow_data_3d,
    fast_slow_linear=fast_model_slow_data_linear,
    slow_slow_linear=slow_model_slow_data_linear,
)

In [6]:
import pandas as pd

records = []

for name, data in preds_and_ground_truths.items():
    model_speed = name.split('_')[0]
    model_type = name.split('_')[2]
    data_speed = name.split('_')[1]
    for (gt, pred) in zip(data[:, data.shape[1] - 1], data[:, data.shape[1] - 2]):
        dd = dict()
        dd['model speed'] = model_speed
        dd['model type'] = model_type
        dd['data speed'] = data_speed
        dd['ground_truth_angle'] = gt
        dd['predicted_angle'] = pred
        records.append(dd)
df = pd.DataFrame.from_records(records)

df.to_csv('gt_and_preds.csv.gz')
df

,model speed,model type,data speed,ground_truth_angle,predicted_angle
0,fast,3d,fast,0.034974,0.452513
1,fast,3d,fast,0.043523,0.473522
2,fast,3d,fast,0.043523,0.482327
3,fast,3d,fast,0.071512,0.474770
4,fast,3d,fast,0.098842,0.491792
...,...,...,...,...,...
158251,slow,linear,slow,0.095727,0.048836
158252,slow,linear,slow,0.109439,0.068060
158253,slow,linear,slow,0.127183,0.073969
158254,slow,linear,slow,0.115749,0.068338


In [7]:
df['mean absolute ground truth angle'] = abs(df.ground_truth_angle)
df['mean absolute predicted angle'] = abs(df.predicted_angle)
df['mean absolute error'] = abs(df.predicted_angle - df.ground_truth_angle)
df = df.drop(columns=['ground_truth_angle', 'predicted_angle'])

In [8]:
df.groupby(['model speed', 'model type', 'data speed']).mean()

mean absolute ground truth angle  \
model speed model type data speed                                     
fast        3d         fast                                0.432614   
                       slow                                0.348772   
            linear     fast                                0.432545   
                       slow                                0.348797   
slow        3d         fast                                0.432614   
                       slow                                0.348772   
            linear     fast                                0.432545   
                       slow                                0.348797   

                                   mean absolute predicted angle  \
model speed model type data speed                                  
fast        3d         fast                             0.418030   
                       slow                             0.350468   
            linear     fast                             0.428431   
                       slow                             0.347928   
slow        3d         fast                             0.341943   
                       slow                             0.328992   
            linear     fast                             0.398105   
                       slow                             0.344026   

                                   mean absolute error  
model speed model type data speed                       
fast        3d         fast                   0.061224  
                       slow                   0.061423  
            linear     fast                   0.023740  
                       slow                   0.026628  
slow        3d         fast                   0.129813  
                       slow                   0.088841  
            linear     fast                   0.047309  
                       slow                   0.023205